In [7]:
from __future__ import division
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import  MiniBatchKMeans
import numpy as np
import pandas as pd


%autosave 0 

Autosave disabled


In [8]:


class HMM (object):
             def __init__(self,A,B,PI):
                 self.A=A
                 self.B=B
                 self.PI=PI   
def viterbi(obs,Modelo1,PI):
    
    delta=np.zeros((len(obs)+1,len(Modelo1.A)))
    phi=np.zeros((len(obs)+1,len(A)))+666
    path =np.zeros(len(obs)+1)
    T=len(obs)
    Modelo1.PI = PI
    delta[0,:]= Modelo1.PI * Modelo1.B[:,obs[0]]
    phi[0,:]=666
    for t in range(len(obs)):
        for j in range(delta.shape[1]):

            delta [t+1,j]=np.max(delta[t] * A[:,j]) * B[j,obs[t]]
            phi[t+1,j]= np.argmax(delta[t] * A[:,j])
    path[T]=int(np.argmax(delta[T,:]))
    for i in np.arange(T-1,0,-1):
        #print (i,phi[i+1,int(path[i+1])])
        path[i]=phi[i+1,int(path[i+1])]
    return(path)
def cuantizar_xy(xy, cc):
    xycuant=cc
    out=np.power(xycuant-xy,2).sum(axis=1).argmin()
    return out
    
def path_to_xy(path,ccxy):
    estimated= pd.DataFrame(path.astype(int).T)
    estimated.columns=[['Path_vit']]
    estimated['xcuant'] = estimated['Path_vit'].apply(lambda x: ccxy[x,0])
    estimated['ycuant'] = estimated['Path_vit'].apply(lambda x: ccxy[x,1])                                
    return (estimated)
def quantized(xyth,ccxyth):
    xythcuant=np.argmin(np.linalg.norm(xyth-ccxyth,axis=1))
    x,y=ccxyth[xythcuant,:2]
    return ((x,y),(xythcuant))



In [10]:
def Markov_A_2_grafo(A,ccxyth):
    dists=np.zeros(A.shape)
    for i in range(A.shape[0]):
        for j in range (A.shape[1]):
            if A[i,j]!=0 :
                dists[i,j]= np.linalg.norm(ccxyth[i]-ccxyth[j])    
    
    
    con = np.where(dists==0,np.inf,dists)
    graphe2=grafo(ccxyth,con)
    return graphe2


class node(object):
    def __init__(self,x,y):
        self.x=x
        self.y=y
        
        
class grafo (object):
             def __init__(self,nodos,conec):
                 self.nodos=nodos
                 self.conec=conec        

def dijkstra(nodoinicial,nodofinal,graphe):
    

    numnodos= len(graphe.nodos)
    con = graphe.conec
    D= np.ones(numnodos)*np.inf
    Prv= np.ones(numnodos)*np.inf
    V= np.zeros(numnodos)
    a = nodoinicial
    D[a]=0
    Prv[a]=0
    Prv[np.where(con[a]!=np.inf)]=a
    V[a]=1
    Dacc=D[a]
    ########
    D=np.minimum(D,con[a]+D[a])
    cont=0
    sucess=False
    while(sucess==False):
        a = np.argmin(D+np.where (V==1,np.inf, V))
        Dacc=D[a]
        Prv[np.where(D>(con[a]+Dacc) )]=a
        V[a]=1
        D=np.minimum(D,con[a]+Dacc)
        if (a== nodofinal):
            print("RUTA CALCULADA ")
            sucess=True
    rutainv=[]
    rutainv.append(nodofinal)
    while(rutainv[-1]!=nodoinicial):
        prv=Prv[int(rutainv[-1])]
        rutainv.append(prv)

    ruta=[]
    for n in reversed(rutainv):
        ruta.append(n)
    return(ruta)


In [12]:
only_nice=True
data=pd.read_csv('lecs_odom(goodman).txt')
#data=data.iloc[:5000,:]

cols=['x','y','theta']
n_reads=data.shape[1]-3
n_reads
etiquetas=[]
for i in range (n_reads):
    etiquetas.append('R'+str(i))

        
for lab in cols:
    etiquetas.append(lab)


data.columns=etiquetas

if (only_nice!=True):
    print("NOT ONLY NICE")
    data2=pd.read_csv('lecturasconodometria.txt')
    data2.columns=etiquetas
    data3= pd.concat((data,data2))
    data=data3






####
#index = data[data.y<-1].index
#data=data.drop(index)
data.describe()



#####

xyth=data[['x','y','theta']]

lecs=data.iloc[:,0:n_reads]


data.describe()


,R0,R1,R2,R3,R4,R5,R6,R7,R8,R9,...,R714,R715,R716,R717,R718,R719,R720,x,y,theta
count,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,...,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000,14136.000000
mean,2.373057,2.377851,2.377058,2.377344,2.374900,2.372769,2.369163,2.367925,2.363259,2.360994,...,2.096605,2.095325,2.102504,2.107841,2.110094,2.114183,2.118283,1.356175,2.452435,0.144713
std,2.612189,2.612958,2.610579,2.612961,2.608324,2.608267,2.605071,2.603893,2.600871,2.597608,...,2.133515,2.128783,2.146245,2.161205,2.165778,2.176601,2.187231,0.774155,1.350172,1.878428
min,0.426443,0.426465,0.426445,0.424923,0.424836,0.424520,0.423348,0.423476,0.422594,0.422392,...,0.391625,0.363851,0.359515,0.355293,0.351181,0.348181,0.349226,-0.196506,-0.069274,-3.141372
25%,1.244837,1.244395,1.243781,1.242657,1.242888,1.241186,1.240283,1.240331,1.239902,1.241728,...,1.216368,1.216115,1.216197,1.218699,1.219602,1.220703,1.224061,0.842793,1.006371,-1.528835
50%,1.752799,1.754201,1.755181,1.756971,1.757669,1.757518,1.757012,1.751673,1.745272,1.746330,...,1.716429,1.719303,1.721844,1.723620,1.728746,1.730286,1.731057,1.247488,2.793368,0.167726
75%,2.489995,2.485551,2.478290,2.470875,2.463181,2.455865,2.448519,2.441354,2.434042,2.426596,...,2.240525,2.242569,2.246129,2.251352,2.257604,2.263259,2.267050,2.063946,3.741621,1.729399
max,13.500000,13.500000,13.500000,13.500000,13.500000,13.500000,13.500000,13.500000,13.500000,13.500000,...,13.500000,13.500000,13.500000,13.500000,13.500000,13.500000,13.500000,2.773765,4.515830,3.141566


In [13]:
get_new_ccs= False
if get_new_ccs:
    
    K_corpus=256
    N=100
    
    kmedias=MiniBatchKMeans(init='k-means++', n_clusters=K_corpus, batch_size=1000,
                      n_init=10, max_no_improvement=10, verbose=0)
    kmedias.fit(lecs)
    ccvk=kmedias.cluster_centers_
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=N, batch_size=1000,
                      n_init=10, max_no_improvement=10, verbose=0)
    mbk.fit(xyth)
    ccxyth=mbk.cluster_centers_
else:
    ccxyth,ccvk=np.load('ccxyth.npy'),np.load('ccvk.npy')
    print("CUANTIZANDO SIN RECALCULAR CC")
aus=[]
auvks=[]
for i,lec in zip(xyth.values, lecs.values):
    _, au= quantized(i , ccxyth)
    aus.append(au)

    auvk= np.power(lec.T-ccvk,2).sum(axis=1,keepdims=True).argmax()
    auvks.append(auvk)
print(i ,au)
data['Y']=aus
data['Vk']=auvks
obs,est= data['Vk'].values,data['Y'].values
data['Y'].describe()   

CUANTIZANDO SIN RECALCULAR CC
[-3.47380805e-04  1.36747526e-01 -2.54557115e+00] 15


count    14136.000000
mean        14.661573
std          9.239864
min          0.000000
25%          7.000000
50%         12.000000
75%         23.000000
max         31.000000
Name: Y, dtype: float64

In [14]:
####BAUM WELCH APPROX###############???!?!?!


N=est.max()+1 
K_corpus= obs.max()+1
TRANS=np.zeros((N,N))
for i in range (N):
    fromTO=[]


    indices=np.asarray(np.where(est==i)).ravel()
   

    for indice in indices:

        if (int(indice)==len(est)-1):
            
            indice=indice-1

        fromTO.append(est[indice+1])
    fromTO.append(0)
    fromTO.append(N-1)
    aux=np.asarray(np.bincount(fromTO    ))
    aux[0]=aux[0]-1
    aux[N-1]=aux[N-1]-1
    
    TRANS[i,:]=aux
TRANS=TRANS/TRANS.sum(axis=1)
#####################################3
EMIS= np.zeros((N,K_corpus))
for Vk in range (K_corpus):

    indices=np.asarray(np.where(obs==Vk)).ravel()
    estconvk= est[indices]

    numestenvk=np.bincount(estconvk)
    if (N-len(numestenvk)>0):
        numestenvk=np.append(numestenvk, np.zeros(N-len(numestenvk)))
    EMIS[:,Vk]=(numestenvk/np.bincount(est))

A,B=TRANS,EMIS
PI=np.ones(N)/N
Modelo1=HMM(A,B,PI)
Modelo2= HMM(A,B,PI)
Modelo2.B=B+.0000001
Modelo2.A=A+.0000001





A.shape , EMIS

/home/oscar/tf/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/home/oscar/tf/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


((32, 32),
 array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.03663793],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]))

In [15]:
def forw_alg(o_k,Modelo):
    PI=Modelo.PI
    K= len(o_k)   #Secuencia Observaciones
    N= len(Modelo.A)  #número de estados
    alpha=np.zeros((N,K))
    alpha[:,0]= PI
    
    for k in range(1,K):
        for j in range (N):
            alpha_k= alpha[:,k-1]
            a= Modelo.A[:,j]
            b= Modelo.B[j,o_k[k]]
            alpha[j,k]=b*np.dot(alpha_k,a)
        #norm=np.linalg.norm(alpha[:,k])
        norm=np.sum(alpha[:,k])
        #alpha[:,k]/= norm
            
                #alpha[k,i]=  np.sum( alpha[:,i-1]*Modelo.A[i,:] *   Modelo.B[:,o_k[k]] )
    
    
    

    
    
    
    
    
    return alpha
def  backw_alg(o_k,Modelo):
    PI=Modelo.PI
    K= len(o_k)   #Secuencia Observaciones
    N= len(Modelo.A)  #número de estados
    beta=np.zeros((N,K))
    beta[:,-1]=1
    
    for t in range(K-1,-1,-1):
        for i in range(N):
            
            beta_t1=beta[:,t+1]
            a= Modelo.A[i,:]
            b= Modelo.B[i,o_k[t]]
            beta[i,t]= b*np.dot(a,beta_t1)
       
        
            

    
    
    
    return beta
    

In [16]:
def forw_alg_norm(o_k,Modelo):
    #MATRIX NOTATION
    PI=Modelo.PI
    K= len(o_k)   #Secuencia Observaciones
    N= len(Modelo.A)  #número de estados
    alpha=np.zeros((N,K))
    c_k= np.zeros(K)
    alpha[:,0]= PI
    c_k[0]=1
    for k in range(1,K):
        alpha_k= alpha[:,k-1]
        a= Modelo.A[:,:]
        b= Modelo.B[:,o_k[k]]
        alpha[:,k]=(b*np.dot(alpha_k,a))* c_k[k-1]
        c_k[k]=1/alpha[:,k].sum()
    return alpha #,c_k
def  backw_alg(o_k,Modelo):
    #MATRIX NOTATION

    PI=Modelo.PI
    K= len(o_k)   #Secuencia Observaciones
    N= len(Modelo.A)  #número de estados
    beta=np.zeros((N,K))
    beta[:,-1]=1
    for t in range(K-2,-1,-1):
        beta_t1=beta[:,t+1]
        beta_t1
        a= Modelo.A[:,:]
        b= Modelo.B[:,o_k[t]]
        beta[:,t]= b*np.dot(a,beta_t1)
    return beta
    

In [17]:
#CALL BAUM WELCH
A,B

(array([[0.98922414, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.99175258, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.99055118, ..., 0.        , 0.00157978,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.98873239, 0.        ,
         0.        ],
        [0.        , 0.        , 0.00629921, ..., 0.        , 0.98894155,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.975     ]]),
 array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.03663793],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0. 

In [18]:
o_k= obs[:30]
alpha,beta=forw_alg(o_k,Modelo1),backw_alg(o_k,Modelo1)

In [19]:
alpha.shape,beta.shape


((32, 30), (32, 30))

PR(O|Modelo) =  P  ; likelihood Lo(M)

In [20]:
T=len(o_k)
t=len(o_k)-1
P= alpha[:,t].sum()
P

nan

gamma_ij=E{ transitions from state i to j conditioned to O}

In [21]:
gamma=np.zeros((N,N,T))
for t in range(T-1):
    for j in range(N):
        for i in range(N):
            a=Modelo1.A[i,j]
            b=Modelo1.B[j,o_k[t]]
            gamma[i,j,t]=alpha[i,t]*a*b*beta[i,t+1]
gamma=np.sum(gamma,axis=2)/P
gamma

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

gamma_i=E{Num of trans out of gamma_i out of ith state given O} = SUM j gamma_ij

In [22]:
gamma=np.sum(alpha*beta ,axis=1)/P
gamma

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan])

In [23]:
A

array([[0.98922414, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.99175258, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.99055118, ..., 0.        , 0.00157978,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.98873239, 0.        ,
        0.        ],
       [0.        , 0.        , 0.00629921, ..., 0.        , 0.98894155,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.975     ]])

In [24]:
lam=.5
eta=.5
W,V=np.zeros(A.shape),np.zeros(B.shape)
DW,DV=np.zeros(A.shape),np.zeros(B.shape)

In [25]:
A_gd=np.exp(lam*W)
for i in range(N):
    aux=A_gd[i,:].sum()
    A_gd[i,:]=A_gd[i,:]/aux


B_gd= np.exp(lam*V)
for i in range(N):
    aux=B_gd[i,:].sum()
    B_gd[i,:]=B_gd[i,:]/aux


A_gd , B_gd

(array([[0.03125, 0.03125, 0.03125, ..., 0.03125, 0.03125, 0.03125],
        [0.03125, 0.03125, 0.03125, ..., 0.03125, 0.03125, 0.03125],
        [0.03125, 0.03125, 0.03125, ..., 0.03125, 0.03125, 0.03125],
        ...,
        [0.03125, 0.03125, 0.03125, ..., 0.03125, 0.03125, 0.03125],
        [0.03125, 0.03125, 0.03125, ..., 0.03125, 0.03125, 0.03125],
        [0.03125, 0.03125, 0.03125, ..., 0.03125, 0.03125, 0.03125]]),
 array([[0.00934579, 0.00934579, 0.00934579, ..., 0.00934579, 0.00934579,
         0.00934579],
        [0.00934579, 0.00934579, 0.00934579, ..., 0.00934579, 0.00934579,
         0.00934579],
        [0.00934579, 0.00934579, 0.00934579, ..., 0.00934579, 0.00934579,
         0.00934579],
        ...,
        [0.00934579, 0.00934579, 0.00934579, ..., 0.00934579, 0.00934579,
         0.00934579],
        [0.00934579, 0.00934579, 0.00934579, ..., 0.00934579, 0.00934579,
         0.00934579],
        [0.00934579, 0.00934579, 0.00934579, ..., 0.00934579, 0.00934579,
    

In [26]:
Modelo_gd=HMM(A_gd,B_gd,PI)

In [27]:
o_k= obs[60:90]
alpha,beta=forw_alg(o_k,Modelo_gd),backw_alg(o_k,Modelo_gd)
LoM = alpha[:,-1].sum()

In [28]:
P=LoM
P

1.405628153554109e-59

In [29]:
gamma=np.zeros((N,N,T))
for t in range(T-1):
    for j in range(N):
        for i in range(N):
            a=Modelo1.A[i,j]
            b=Modelo1.B[j,o_k[t]]
            gamma[i,j,t]=alpha[i,t]*a*b*beta[i,t+1]
gamma=np.sum(gamma,axis=2)/P
gamma

array([[ 7.2712771 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , 10.21517935,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  4.97605726, ...,  0.        ,
         0.03184452,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  8.88449167,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.03164424, ...,  0.        ,
        19.93467128,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [30]:
n_ij_o, n_i_o=gamma , gamma.sum(axis=1)

In [31]:
for i in range (N):
    for j in range (N):
         DW[i,j]=  (eta)* (  n_ij_o[i,j] -(n_i_o[i]* A_gd[i,j]))

In [32]:
DW

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [33]:
W=W + DW
W

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [ ]:
lam=.5
eta=1.5

W,V=np.zeros(A.shape),np.zeros(B.shape)
DW,DV=np.zeros(A.shape),np.zeros(B.shape)

for k in range((int)(len(obs)/30)):
    o_k=obs[30*k:30*k+30]
    A_gd=np.exp(lam*W)
    for i in range(N):
        aux=A_gd[i,:].sum()
        A_gd[i,:]=A_gd[i,:]/aux
    B_gd= np.exp(lam*V)
    for i in range(N):
        aux=B_gd[i,:].sum()
        B_gd[i,:]=B_gd[i,:]/aux
    Modelo_gd=HMM(A_gd,B_gd,PI)
    alpha,beta = forw_alg(o_k,Modelo_gd), backw_alg(o_k,Modelo_gd)
    P = alpha[:,-1].sum()
    gamma=np.zeros((N,N,T))
    for t in range(T-1):
        for j in range(N):
            for i in range(N):
                a=Modelo1.A[i,j]
                b=Modelo1.B[j,o_k[t]]
                gamma[i,j,t]=alpha[i,t]*a*b*beta[i,t+1]
    gamma=np.sum(gamma,axis=2)/P
    n_ij_o, n_i_o=gamma , gamma.sum(axis=1)
    for i in range (N):
        for j in range (N):
             DW[i,j]=  (eta)* (  n_ij_o[i,j] -(n_i_o[i]* A_gd[i,j]))
    W=W + DW
    print(DW)

                
    
    
    
    

    

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan

In [ ]:
A , 

In [ ]:
A_gd